# Region Detection Comparing different Segmentation Methods and Hough Circle Detection

This program defines the functions for threshold, K-Means, DBSCAN, and watershed methods for detecting ice void regions in the PMCs. The Hough Transform is passed on the boundaries of each method. 

In [ ]:
# Function to detect regions using DBSCAN clustering
def detect_regions_dbscan(cloud_albedo, eps=0.35, min_samples=15):
    albedo_flat = cloud_albedo.flatten()  # Flatten the 2D albedo array to 1D
    coordinates = np.array([(i, j) for i in range(cloud_albedo.shape[0]) for j in range(cloud_albedo.shape[1])])  # Create a list of (row, column) coordinate pairs
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(coordinates, sample_weight=albedo_flat)  # Apply DBSCAN clustering with given parameters
    labels = clustering.labels_.reshape(cloud_albedo.shape)  # Reshape the 1D label array back to 2D
    regions_map = (labels == -1).astype(int)  # Convert void regions (label -1) to 1, others to 0
    return regions_map  # Return the binary regions map

In [ ]:
# Function to detect regions using a threshold method
def detect_regions_threshold(cloud_albedo, threshold=2, min_region_size=100, max_region_size=50000):
    binary_albedo = np.where(cloud_albedo > threshold, 0, 1)  # Binarize albedo data based on threshold
    dilated_albedo = ndimage.binary_dilation(binary_albedo, structure=np.ones((3, 3)))  # Dilate the binary image to fill small gaps
    border = dilated_albedo - binary_albedo  # Identify the border regions
    surrounded_regions = dilated_albedo - border  # Extract the regions fully surrounded by others
    labeled_regions, num_regions = ndimage.label(surrounded_regions)  # Label the connected regions
    region_sizes = ndimage.sum(surrounded_regions, labeled_regions, range(1, num_regions + 1))  # Calculate the size of each region
    region_mask = np.isin(labeled_regions, np.where((region_sizes > min_region_size) & (region_sizes < max_region_size))[0] + 1)  # Create a mask for regions within the size limits
    return region_mask  # Return the mask of valid regions

In [ ]:
# Function to detect regions using the watershed algorithm
def detect_watershed_albedo_(cloud_albedo, min_region_size=100, max_region_size=250000):
    gradient = sobel(cloud_albedo)  # Calculate the gradient magnitude of the albedo image
    local_min = local_minima(cloud_albedo)  # Detect local minima in the albedo image
    markers = label(local_min)  # Label the local minima markers
    labels = watershed(cloud_albedo, markers=markers)  # Apply the watershed algorithm using the gradient and markers
    valid_labels = [region.label for region in regionprops(labels) if min_region_size <= region.area <= max_region_size]  # Filter regions based on size
    region_mask = np.isin(labels, valid_labels)  # Create a mask for valid regions
    return labels  # Return the labeled regions

In [ ]:
# Function to detect regions using KMeans clustering
def detect_regions_kmeans(cloud_albedo, n_clusters=4):
    albedo_flat = cloud_albedo.flatten()  # Flatten the 2D albedo array to 1D
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(albedo_flat.reshape(-1, 1))  # Apply KMeans clustering with given number of clusters
    cluster_labels = kmeans.labels_ + 1  # Adjust labels to avoid 0 as background
    cluster_labels_2d = cluster_labels.reshape(cloud_albedo.shape)  # Reshape the 1D label array back to 2D
    return cluster_labels_2d  # Return the 2D array of cluster labels

In [ ]:
# Function to plot albedo data with detected regions and contours
def plot_albedo_with_regions_contours(cloud_albedo, regions_map, title='', min_size=100, max_size=250000, method_name=''):
    plt.figure(figsize=(16, 10))  # Set the figure size
    plt.rcParams.update({'font.size': 20})  # Update the font size
    
    # Plot the original albedo data
    plt.subplot(3, 1, 1)
    plt.imshow(cloud_albedo, cmap='Blues_r')  # Display albedo data with 'Blues_r' colormap
    plt.colorbar(label='Cloud Albedo')  # Add colorbar with label
    plt.title('Cloud Albedo')  # Add title
    plt.xlabel('X Pixel')  # Add X-axis label
    plt.ylabel('Y Pixel')  # Add Y-axis label
    
    # Plot the albedo data with detected regions
    plt.subplot(3, 1, 2)
    plt.imshow(cloud_albedo, cmap='Blues_r')  # Display albedo data again
    plt.colorbar(label='Cloud Albedo')  # Add colorbar with label
    plt.title('Detected Regions')  # Add title
    plt.xlabel('X Pixel')  # Add X-axis label
    plt.ylabel('Y Pixel')  # Add Y-axis label
    
    labeled_regions = label(regions_map)  # Label the detected regions
    props = regionprops(labeled_regions)  # Get properties of labeled regions
    
    # List of colors for contours
    colors = [
        '#FF00FF', '#00FFFF', '#FFFF00', '#00FF00', '#FF0000', '#0000FF',
        '#FF6600', '#CCFF00', '#FF33CC', '#33FF33', '#6633FF', '#FF3399'
    ]
    
    # Plot the contours of each region
    for i, prop in enumerate(props):
        if prop.area >= min_size:  # Only plot regions larger than min_size
            color = colors[i % len(colors)]  # Cycle through the list of colors
            contours = find_contours(labeled_regions == prop.label, 0.5)  # Find contours for the region
            for contour in contours:
                plt.plot(contour[:, 1], contour[:, 0], color=color)  # Plot the contour with the specified color
    
    # Plot the albedo data with Hough Transform circles
    plt.subplot(3, 1, 3)
    plt.imshow(cloud_albedo, cmap='gray')  # Display albedo data with 'gray' colormap
    plt.colorbar(label='Cloud Albedo')  # Add colorbar with label
    plt.title('Detected Regions with Hough Transform Circles')  # Add title
    plt.xlabel('X Pixel')  # Add X-axis label
    plt.ylabel('Y Pixel')  # Add Y-axis label
    
    edges = np.zeros_like(labeled_regions, dtype=np.uint8)  # Create an empty array for edges
    
    # Find contours and edges for regions within size limits
    for i, prop in enumerate(props):
        if min_size <= prop.area <= max_size:  # Only consider regions within size limits
            region_mask = (labeled_regions == prop.label)  # Create a mask for the region
            contours = find_contours(region_mask, level=0.5)  # Find contours for the region
            for contour in contours:
                contour = np.round(contour).astype(int)  # Round contour coordinates to integers
                edges[contour[:, 0], contour[:, 1]] = 1  # Mark edges in the array
                
    # Apply Hough Transform to find circles
    if np.any(edges):  # Only if there are edges detected
        hough_radii = np.arange(20, 300)  # Define a range of radii to search for circles
        hough_res = hough_circle(edges.astype(np.uint8), hough_radii)  # Apply Hough Transform
        accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, min_xdistance=20, min_ydistance=20, total_num_peaks=5)  # Find peaks in the Hough space
        for center_y, center_x, radius in zip(cy, cx, radii):  # Plot each detected circle
            circle = Circle((center_x, center_y), radius=radius, fill=False, color='red', linewidth=2)  # Create a red circle
            plt.gca().add_patch(circle)  # Add the circle to the plot
    
    plt.suptitle(title)  # Add the overall title
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()  # Show the plot

In [ ]:
import os
os.environ['PROJ_LIB'] = '/mnt/home/anaise.gaillard/.conda/envs/watershed/share/proj'  # Set the environment variable for PROJ library path

# Function to unzip a gzipped file
def gunzip_shutil(source_filepath, dest_filepath, block_size=1024*1024):
    with gzip.open(source_filepath, 'rb') as s_file, open(dest_filepath, 'wb') as d_file:  # Open the source file for reading and destination file for writing
        shutil.copyfileobj(s_file, d_file, block_size)  # Copy content from source to destination in chunks of block_size

In [ ]:
# Dictionary of orbit files with paths to compressed and uncompressed files
orbits_dic = {
    '01292': ('/data/aim/interim_archive/cips/data/PMC/north_2007/level_2/ver_05.20/rev_05/cips_sci_2_orbit_01292_2007-202_v05.20_r05_cat.nc.gz', 
              '/data/aim/interim_archive/cips/data/PMC/north_2007/level_2/ver_05.20/rev_05/cips_sci_2_orbit_01292_2007-202_v05.20_r05_cld.nc.gz'),
    '01269': ('/data/aim/interim_archive/cips/data/PMC/north_2007/level_2/ver_05.20/rev_05/cips_sci_2_orbit_01269_2007-200_v05.20_r05_cat.nc.gz', 
              '/data/aim/interim_archive/cips/data/PMC/north_2007/level_2/ver_05.20/rev_05/cips_sci_2_orbit_01269_2007-200_v05.20_r05_cld.nc.gz'),
    '01263': ('/data/aim/interim_archive/cips/data/PMC/north_2007/level_2/ver_05.20/rev_05/cips_sci_2_orbit_01263_2007-200_v05.20_r05_cat.nc.gz', 
              '/data/aim/interim_archive/cips/data/PMC/north_2007/level_2/ver_05.20/rev_05/cips_sci_2_orbit_01263_2007-200_v05.20_r05_cld.nc.gz'),
    '01240': ('/data/aim/interim_archive/cips/data/PMC/north_2007/level_2/ver_05.20/rev_05/cips_sci_2_orbit_01240_2007-199_v05.20_r05_cat.nc.gz', 
              '/data/aim/interim_archive/cips/data/PMC/north_2007/level_2/ver_05.20/rev_05/cips_sci_2_orbit_01240_2007-199_v05.20_r05_cld.nc.gz'),
    '00814': ('/data/aim/interim_archive/cips/data/PMC/north_2007/level_2/ver_05.20/rev_05/cips_sci_2_orbit_00814_2007-170_v05.20_r05_cat.nc.gz', 
              '/data/aim/interim_archive/cips/data/PMC/north_2007/level_2/ver_05.20/rev_05/cips_sci_2_orbit_00814_2007-170_v05.20_r05_cld.nc.gz')
}

# Dictionary of methods to detect regions
methods = {
    "DBSCAN": detect_regions_dbscan,
    "KMeans": detect_regions_kmeans,
    "Threshold": detect_regions_threshold,
    "Watershed": detect_watershed_albedo_
}

In [ ]:
# Process each orbit
for orbit, (catfile, cldfile) in orbits_dic.items():
    # Unzip files
    gunzip_shutil(catfile, 'geolocation.nc')  # Unzip the catalog file
    gunzip_shutil(cldfile, 'cloud.nc')  # Unzip the cloud file
    
    # Load cloud data from the unzipped file
    cloud_data = nc.Dataset('cloud.nc')  # Open the NetCDF file
    cloud_albedo = cloud_data['CLD_ALBEDO'][:]  # Extract the albedo data
    cloud_albedo_nan = np.nan_to_num(cloud_albedo, nan=-1000)  # Replace NaNs with -1000
    
    # Apply each method to detect regions and plot results
    for method_name, method_func in methods.items():  # Iterate over each method
        regions_map = method_func(cloud_albedo_nan)  # Detect regions using the current method
        plot_albedo_with_regions_contours(cloud_albedo, regions_map, title=f'Orbit {orbit} - Method {method_name}', min_size=100, max_size=250000)  # Plot the results